# Combining cow with other attributes

In [1]:
import cudf
import pyarrow as pa
import pandas as pd
import numpy as np
import cupy as cp
import os

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [3]:
df = cudf.read_parquet('../data/all_states_lat_long_income_education_age_sex.parquet/*')

OSError: ../data/all_states_lat_long_income_education_age_sex.parquet/* could not be resolved to any files

In [ ]:
df = df.sort_values('GISJOIN')
df

In [ ]:
df.dtypes

In [ ]:
df_cow = cudf.read_parquet('./cow_step_2.parquet/*')

In [ ]:
df_cow.cow = df_cow.cow.astype('int8')
df_cow = df_cow.sort_values('GISJOIN').reset_index()
df_cow.drop_column('index')

In [ ]:
df_cow.dtypes

In [ ]:
df_cow

In [ ]:
df_cow_with_gis_mappings = cudf.read_parquet('./GISJOIN_to_GISJOIN_og_mapping.parquet/*')
df_cow_with_gis_mappings.age = df_cow_with_gis_mappings.age.astype('int8')

In [ ]:
df_cow_with_gis_mappings = df_cow_with_gis_mappings.sort_values('GISJOIN').reset_index()
df_cow_with_gis_mappings.drop_column('index')

In [ ]:
df_cow_with_gis_mappings.drop_column('GISJOIN_og')

In [ ]:
df_cow_with_gis_mappings

In [ ]:
df_cow_with_gis_mappings['GISJOIN_cow'] = df_cow.GISJOIN.values

In [ ]:
df_cow_with_gis_mappings.query('GISJOIN != GISJOIN_cow')

In [ ]:
df_cow_with_gis_mappings['cow'] = df_cow.cow.values

In [ ]:
df_cow_with_gis_mappings.drop_column('GISJOIN_cow')
df_cow_with_gis_mappings.drop_column('GISJOIN')

In [ ]:
df_cow_with_gis_mappings.cow.value_counts()

In [ ]:
del(df_cow)

### merge sex and cow with lat-long table

In [ ]:
df_cow_with_gis_mappings

In [ ]:
df = df.merge(df_cow_with_gis_mappings, on='p_id', how='left')

In [ ]:
df

In [ ]:
df.drop_column('age_y')
df.drop_column('sex_y')

In [ ]:
df.cow = df.cow.fillna(8)
df.rename({
    'sex_x':'sex',
    'age_x': 'age'
}, inplace=True)

In [ ]:
df.dtypes

In [ ]:
df.cow.value_counts()

In [ ]:
del(df_cow_with_gis_mappings)

In [ ]:
df

In [ ]:
df.to_parquet('../data/all_states_lat_long_cow_income_education_age_sex.parquet')